<a href="https://colab.research.google.com/github/mnpro98/LN-Actividad-Integradora-3/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Incluir la funcionalidad de buscar posts en Reddit por usuarios. A las publicaciones recolectadas 
#de  Reddit  se  les  debe  hacer  un  procesamiento  para  eliminar  hyperlinks,  menciones  a  otros 
#subreddits (r/smashbros) y menciones a otros usuarios (/Username). (10 puntos) 

In [ ]:
!pip install praw

  Using cached praw-7.4.0-py3-none-any.whl (167 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
     |████████████████████████████████| 52 kB 934 kB/s 


In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Activation, Dense, Dropout, Embedding, Flatten, Conv1D, MaxPooling1D, LSTM, GRU
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
! wget nlp.stanford.edu/data/glove.6B.zip
! unzip glove.6B.zip

--2021-10-31 19:23:11--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-10-31 19:23:11--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-10-31 19:23:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [ ]:
# Initialize dictionary to be converted to dataframe
data_dic = {
      'text': [],
      'classification': [],
}

In [ ]:
def fill_dic(file_name, classification):
  with open(file_name, encoding='latin-1') as f:
    lines = f.readlines()
    for line in lines:
      data_dic['text'].append(line)
      data_dic['classification'].append(classification)

In [ ]:
fill_dic('test_neg.txt', 0)
fill_dic('train_neg.txt', 0)
fill_dic('test_pos.txt', 1)
fill_dic('train_pos.txt', 1)

In [ ]:
reviews_df = pd.DataFrame(data=data_dic)
reviews_df

,text,classification
0,Once again Mr. Costner has dragged out a movie...,0
1,This is an example of why the majority of acti...,0
2,"First of all I hate those moronic rappers, who...",0
3,Not even the Beatles could write songs everyon...,0
4,Brass pictures (movies is not a fitting word f...,0
...,...,...
49201,"Seeing as the vote average was pretty low, and...",1
49202,"The plot had some wretched, unbelievable twist...",1
49203,I am amazed at how this movie(and most others ...,1
49204,A Christmas Together actually came before my t...,1


In [ ]:
def clean_df(df):
  unwanted_ss = ['<br />', 'http\S+|www.\S+']
  stopwords = ["i", "I", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "only", "own", "same", "so", "than", "too", "very", "s", "can", "will", "just", "should", "now"]
  joined = unwanted_ss + stopwords
  df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (joined)]))
  return df

In [ ]:
reviews_df_clean = clean_df(reviews_df)
reviews_df_clean

,text,classification
0,Once Mr. Costner dragged movie far longer nece...,0
1,This example majority action films same. Gener...,0
2,"First hate moronic rappers, could'nt act gun p...",0
3,Not even Beatles could write songs everyone li...,0
4,Brass pictures (movies not fitting word them) ...,0
...,...,...
49201,"Seeing vote average pretty low, fact clerk vid...",1
49202,"The plot wretched, unbelievable twists. Howeve...",1
49203,amazed movie(and others average 5 stars lower ...,1
49204,"A Christmas Together actually came time, I've ...",1


In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(reviews_df_clean['text'])
tokenizer.texts_to_sequences(reviews_df_clean['text'])
pad_sequences(tokenizer.texts_to_sequences(reviews_df_clean['text']))

array([[    0,     0,     0, ...,  1609,   475,    30],
       [    0,     0,     0, ...,   105,   396,    82],
       [    0,     0,     0, ...,    72,    77,   100],
       ...,
       [    0,     0,     0, ...,  1088,   133, 10237],
       [    0,     0,     0, ...,  8737,  3830,    20],
       [    0,     0,     0, ...,    16,  2423,    34]], dtype=int32)

In [ ]:
print(len(tokenizer.word_index))

118791


In [ ]:
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
embedding_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
embeddings_index = {}
f = open('glove.6B.300d.txt')

for line in f:
  values = line.split()
  word = values[0]
  floats = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = floats
f.close()

In [ ]:
for word, i in tokenizer.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.40287   , -0.48699   ,  0.091598  , ..., -0.66092002,
         0.54541999,  0.28307   ],
       [ 0.04656   ,  0.21318001, -0.0074364 , ...,  0.0090611 ,
        -0.20988999,  0.053913  ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
embedding_layer = Embedding(len(tokenizer.word_index) + 1, 300, weights=[embedding_matrix], input_length=300, trainable=False)

# Architecture 1

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.5))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          35637600  
_________________________________________________________________
dropout_1 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 35,798,101
Trainable params: 160,501
Non-trainable params: 35,637,600
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [ ]:
#from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data = train_test_split(reviews_df_clean, test_size=0.2, 
                                         random_state=7)

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_data.classification.tolist())

y_train = encoder.transform(train_data.classification.tolist())
y_test = encoder.transform(test_data.classification.tolist())

y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)

print("y_train",y_train.shape)
print("y_test",y_test.shape)

y_train (39364, 1)
y_test (9842, 1)


In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data.text), maxlen=300)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data.text), maxlen=300)

In [ ]:
history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_split=0.2)

Epoch 1/10
247/247 [==============================] - 536s 2s/step - loss: 0.4931 - accuracy: 0.7598 - val_loss: 0.4917 - val_accuracy: 0.8194
Epoch 2/10
247/247 [==============================] - 533s 2s/step - loss: 0.4178 - accuracy: 0.8114 - val_loss: 0.3742 - val_accuracy: 0.8548
Epoch 3/10
247/247 [==============================] - 534s 2s/step - loss: 0.3680 - accuracy: 0.8386 - val_loss: 0.3627 - val_accuracy: 0.8618
Epoch 4/10
247/247 [==============================] - 532s 2s/step - loss: 0.3344 - accuracy: 0.8541 - val_loss: 0.3096 - val_accuracy: 0.8753
Epoch 5/10
247/247 [==============================] - 532s 2s/step - loss: 0.3169 - accuracy: 0.8645 - val_loss: 0.2929 - val_accuracy: 0.8806
Epoch 6/10
247/247 [==============================] - 532s 2s/step - loss: 0.2985 - accuracy: 0.8731 - val_loss: 0.3113 - val_accuracy: 0.8791
Epoch 7/10
247/247 [==============================] - 535s 2s/step - loss: 0.2848 - accuracy: 0.8792 - val_loss: 0.2870 - val_accuracy: 0.8868

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=128)
print()
print("ACCURACY:",score[1])
print("LOSS:",score[0])

77/77 [==============================] - 30s 396ms/step - loss: 0.2871 - accuracy: 0.8899

ACCURACY: 0.8898597955703735
LOSS: 0.2870611548423767


In [ ]:
def decode_sentiment(score, include_neutral=True):
    if include_neutral:        
        label = "NEUTRAL"
        if score <= 0.4:
            label = "NEGATIVE"
        elif score >= 0.7:
            label = "POSITIVE"

        return label
    else:
        return 0 if score < 0.5 else 1

In [ ]:
import time

In [ ]:
def predict(text, include_neutral=True):
    start_at = time.time()
    # Tokenize text
    x_test = pad_sequences(tokenizer.texts_to_sequences([text]), maxlen=300)
    # Predict
    score = model.predict([x_test])[0]
    # Decode sentiment
    label = decode_sentiment(score, include_neutral=include_neutral)

    return {"label": label, "score": float(score),
       "elapsed_time": time.time()-start_at}

In [ ]:
predict("I hated this movie, it's awful. I almost threw up.")

{'elapsed_time': 0.09603333473205566,
 'label': 'NEGATIVE',
 'score': 0.07438838481903076}

In [ ]:
predict("That was awesome! I would definitely watch it again.")

{'elapsed_time': 0.09218192100524902,
 'label': 'POSITIVE',
 'score': 0.90274977684021}

In [ ]:
predict("Honestly, it was an okay movie. I liked some parts and disliked others.")

{'elapsed_time': 0.09893321990966797,
 'label': 'NEUTRAL',
 'score': 0.5404224991798401}

In [ ]:
y_pred_1d = []
y_test_1d = list(test_data.classification)
scores = model.predict(x_test, verbose=1, batch_size=8000)
y_pred_1d = [decode_sentiment(score, include_neutral=False) for score in scores]

2/2 [==============================] - 42s 6s/step


In [ ]:
print(len(y_pred_1d))
print(len(y_test_1d))
print(len(scores))

9842
9842
9842


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [ ]:
print(classification_report(y_test_1d, y_pred_1d))
# 0 - Negative, 1 - Positive

              precision    recall  f1-score   support

           0       0.86      0.92      0.89      4791
           1       0.92      0.86      0.89      5051

    accuracy                           0.89      9842
   macro avg       0.89      0.89      0.89      9842
weighted avg       0.89      0.89      0.89      9842



# Architecture 2

In [ ]:
model2 = Sequential()
model2.add(embedding_layer)
model2.add(Dropout(0.5))
model2.add(Dense(60, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
model2.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          35637600  
_________________________________________________________________
dropout_2 (Dropout)          (None, 300, 300)          0         
_________________________________________________________________
dense_16 (Dense)             (None, 300, 60)           18060     
_________________________________________________________________
dense_17 (Dense)             (None, 300, 1)            61        
Total params: 35,655,721
Trainable params: 18,121
Non-trainable params: 35,637,600
_________________________________________________________________


In [ ]:
# Compile model
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history2 = model2.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_split=0.2)

Epoch 1/10
247/247 [==============================] - 53s 214ms/step - loss: 0.6917 - accuracy: 0.5217 - val_loss: 0.6874 - val_accuracy: 0.5274
Epoch 2/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6891 - accuracy: 0.5259 - val_loss: 0.6868 - val_accuracy: 0.5287
Epoch 3/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6885 - accuracy: 0.5268 - val_loss: 0.6868 - val_accuracy: 0.5279
Epoch 4/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6881 - accuracy: 0.5276 - val_loss: 0.6862 - val_accuracy: 0.5296
Epoch 5/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6876 - accuracy: 0.5288 - val_loss: 0.6860 - val_accuracy: 0.5302
Epoch 6/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6874 - accuracy: 0.5294 - val_loss: 0.6858 - val_accuracy: 0.5307
Epoch 7/10
247/247 [==============================] - 52s 212ms/step - loss: 0.6873 - accuracy: 0.5297 - val_loss: 0.6857 - val_ac

In [ ]:
y_pred_1d2 = []
y_test_1d2 = list(test_data.classification)
scores2 = model2.predict(x_test, verbose=1, batch_size=8000)
y_pred_1d2 = [decode_sentiment(score[0], include_neutral=False) for score in scores2]

2/2 [==============================] - 3s 616ms/step


In [ ]:
print(classification_report(y_test_1d2, y_pred_1d2))
# 0 - Negative, 1 - Positive
#print(len(y_test_1d2))
#print(len(y_pred_1d2))

              precision    recall  f1-score   support

           0       0.56      0.03      0.06      4791
           1       0.52      0.98      0.67      5051

    accuracy                           0.52      9842
   macro avg       0.54      0.50      0.37      9842
weighted avg       0.53      0.52      0.37      9842



# Architecture 3

In [ ]:
model3 = Sequential()
model3.add(embedding_layer)
model3.add(Dense(units=8, input_dim=4,
  activation='tanh')) 
model3.add(Dense(units=8, activation='tanh')) 
model3.add(Dense(units=1, activation='sigmoid'))
model3.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          35637600  
_________________________________________________________________
dense_11 (Dense)             (None, 300, 8)            2408      
_________________________________________________________________
dense_12 (Dense)             (None, 300, 8)            72        
_________________________________________________________________
dense_13 (Dense)             (None, 300, 1)            9         
Total params: 35,640,089
Trainable params: 2,489
Non-trainable params: 35,637,600
_________________________________________________________________


In [ ]:
# Compile model
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history3 = model3.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_split=0.2)

Epoch 1/10
247/247 [==============================] - 17s 67ms/step - loss: 0.6900 - accuracy: 0.5219 - val_loss: 0.6880 - val_accuracy: 0.5249
Epoch 2/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6880 - accuracy: 0.5269 - val_loss: 0.6880 - val_accuracy: 0.5253
Epoch 3/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6877 - accuracy: 0.5287 - val_loss: 0.6875 - val_accuracy: 0.5263
Epoch 4/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6875 - accuracy: 0.5290 - val_loss: 0.6875 - val_accuracy: 0.5260
Epoch 5/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6873 - accuracy: 0.5290 - val_loss: 0.6874 - val_accuracy: 0.5262
Epoch 6/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6873 - accuracy: 0.5297 - val_loss: 0.6872 - val_accuracy: 0.5269
Epoch 7/10
247/247 [==============================] - 16s 64ms/step - loss: 0.6871 - accuracy: 0.5301 - val_loss: 0.6869 - val_accuracy:

In [ ]:
y_pred_1d3 = []
y_test_1d3 = list(test_data.classification)
scores3 = model3.predict(x_test, verbose=1, batch_size=8000)
y_pred_1d3 = [decode_sentiment(score[0], include_neutral=False) for score in scores3]

2/2 [==============================] - 2s 320ms/step


In [ ]:
print(classification_report(y_test_1d3, y_pred_1d3))

              precision    recall  f1-score   support

           0       0.57      0.03      0.06      4791
           1       0.52      0.98      0.68      5051

    accuracy                           0.52      9842
   macro avg       0.54      0.50      0.37      9842
weighted avg       0.54      0.52      0.38      9842



In [125]:
![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)(https://colab.research.google.com/drive/1JeLc4vlCCW6ceQTGUUvevU1CI4pnjJ8h?usp=sharing)

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `[Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)(https://colab.research.google.com/drive/1JeLc4vlCCW6ceQTGUUvevU1CI4pnjJ8h?usp=sharing)'
